In [7]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from google.colab import drive

# drive.mount('/content/drive')

# Load dataset images
def load_images(image_paths, label):
    images, labels = [], []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))  # Resize to 128x128
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        images.append(img.flatten())  # Flatten image
        labels.append(label)
    return np.array(images), np.array(labels)

# Define dataset paths
mask_data_dir = "/content/drive/My Drive/VR datasets/Face-Mask-Detection/dataset/with_mask/"
no_mask_data_dir = "/content/drive/My Drive/VR datasets/Face-Mask-Detection/dataset/without_mask"

# Get image file paths
mask_images = glob(os.path.join(mask_data_dir, "*.jpg"))
no_mask_images = glob(os.path.join(no_mask_data_dir, "*.jpg"))

# Load both datasets separately
X_mask, y_mask = load_images(mask_images, label=1)
X_no_mask, y_no_mask = load_images(no_mask_images, label=0)

# Combine both datasets
X = np.concatenate((X_mask, X_no_mask), axis=0)
y = np.concatenate((y_mask, y_no_mask), axis=0)

# Shuffle data
X, y = shuffle(X, y, random_state=42)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Support Vector Machine (SVM)
svm_model = SVC(kernel="rbf", C=10, gamma="scale")  # RBF kernel with adjusted C
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")


# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

# Import PyTorch only for Neural Network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torch
import torch.nn as nn
import torch.optim as optim

# Neural Network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(128 * 128, 224)
        self.fc2 = nn.Linear(224, 224)
        self.fc3 = nn.Linear(224, 224)
        self.fc4 = nn.Linear(224, 224)
        self.fc5 = nn.Linear(224, 224)
        self.fc6 = nn.Linear(224, 2)  # Output layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)  # Dropout with 30% probability

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)  # Dropout after first layer
        x = self.relu(self.fc2(x))
        x = self.dropout(x)  # Dropout after second layer
        x = self.relu(self.fc3(x))
        x = self.dropout(x)  # Dropout after third layer
        x = self.relu(self.fc4(x))
        x = self.dropout(x)  # Dropout after fourth layer
        x = self.relu(self.fc5(x))
        x = self.dropout(x)  # Dropout after fifth layer
        x = self.fc6(x)  # No activation, since CrossEntropyLoss applies Softmax
        return x

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate reduced



# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if epoch%10==0:
      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate model
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f"Neural Network Accuracy: {correct / total:.4f}")


SVM Accuracy: 0.9057
Random Forest Accuracy: 0.8805
Epoch [1/100], Loss: 0.0000
Epoch [11/100], Loss: 0.0013
Epoch [21/100], Loss: 0.0002
Epoch [31/100], Loss: 0.0177
Epoch [41/100], Loss: 1.9169
Epoch [51/100], Loss: 0.0456
Epoch [61/100], Loss: 0.0000
Epoch [71/100], Loss: 0.0000
Epoch [81/100], Loss: 0.0000
Epoch [91/100], Loss: 0.0000
Neural Network Accuracy: 0.8911


In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from google.colab import drive
from skimage.feature import hog  # NEW: Import HOG from scikit-image

drive.mount('/content/drive')

# Load dataset images using HOG feature extraction
def load_images(image_paths, label):
    images, labels = [], []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))  # Resize to 128x128
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        # Extract HOG features
        features = hog(img, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), block_norm='L2-Hys',
                       visualize=False, feature_vector=True)
        images.append(features)
        labels.append(label)
    return np.array(images), np.array(labels)

# Define dataset paths
mask_data_dir = "/content/drive/My Drive/VR datasets/Face-Mask-Detection/dataset/with_mask/"
no_mask_data_dir = "/content/drive/My Drive/VR datasets/Face-Mask-Detection/dataset/without_mask"

# Get image file paths
mask_images = glob(os.path.join(mask_data_dir, "*.jpg"))
no_mask_images = glob(os.path.join(no_mask_data_dir, "*.jpg"))

# Load both datasets separately
X_mask, y_mask = load_images(mask_images, label=1)
X_no_mask, y_no_mask = load_images(no_mask_images, label=0)

# Combine both datasets
X = np.concatenate((X_mask, X_no_mask), axis=0)
y = np.concatenate((y_mask, y_no_mask), axis=0)

# Shuffle data
X, y = shuffle(X, y, random_state=42)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Support Vector Machine (SVM)
svm_model = SVC(kernel="rbf", C=10, gamma="scale")  # RBF kernel with adjusted C
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

# Import PyTorch only for Neural Network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Determine input dimension from HOG features (all images now have the same length)
INPUT_DIM = X_train.shape[1]

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Neural Network with 5 hidden layers, 224 neurons each, and dropout
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(INPUT_DIM, 224)
        self.fc2 = nn.Linear(224, 224)
        self.fc3 = nn.Linear(224, 224)
        self.fc4 = nn.Linear(224, 224)
        self.fc5 = nn.Linear(224, 224)
        self.fc6 = nn.Linear(224, 2)  # Output layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)  # Dropout with 30% probability

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)  # Dropout after first layer
        x = self.relu(self.fc2(x))
        x = self.dropout(x)  # Dropout after second layer
        x = self.relu(self.fc3(x))
        x = self.dropout(x)  # Dropout after third layer
        x = self.relu(self.fc4(x))
        x = self.dropout(x)  # Dropout after fourth layer
        x = self.relu(self.fc5(x))
        x = self.dropout(x)  # Dropout after fifth layer
        x = self.fc6(x)  # No activation, since CrossEntropyLoss applies Softmax
        return x

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate model
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f"Neural Network Accuracy: {correct / total:.4f}")


Mounted at /content/drive
SVM Accuracy: 0.9336
Random Forest Accuracy: 0.8805
Epoch [1/100], Loss: 0.0029
Epoch [11/100], Loss: 0.0000
Epoch [21/100], Loss: 0.0000
Epoch [31/100], Loss: 0.0000
Epoch [41/100], Loss: 0.0000
Epoch [51/100], Loss: 0.0000
Epoch [61/100], Loss: 0.0000
Epoch [71/100], Loss: 0.0000
Epoch [81/100], Loss: 0.0000
Epoch [91/100], Loss: 0.0000
Neural Network Accuracy: 0.9110


The chosen models effectively handle high-dimensional data and mitigate the impact of irrelevant features. Since this is a supervised learning task, KNN and GMM were not considered. Although logistic regression, a linear classifier, was included, it exhibited relatively poor performance. Traditional ML methods, such as SVM and random forests, showed varying results, with random forests demonstrating greater robustness due to its ensemble nature. As expected, the neural network outperformed traditional models, leveraging its ability to learn hierarchical representations.Also,intead of simply flattening the image ,if we use handcrafted features like HoG,performance improves